In [1]:
import os
import json
from ultralytics import YOLO
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="expandable_segments:True"

# Initial Training

In [ ]:
# Train the model
os.chdir("/data/mtmoore/school/CSiML_AI395T/final_project/dataset/yolov8_dataset/")
project="/data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative"
batchsizes={ 2880: 1, 1440: 4, 720: 16, 640: 32 }
model_name="11s"

for model_name in [ "11n", "11s", "11s" ]:
    for imgsize in [720, 1440, 2880]:
        for i in list(range(1, 25)) + list(range(25, 50, 5)) + list(range(60, 101, 10)):
            
            name = f"{model_name}_{imgsize}imgsz_IP8M-H-SW-{i}epochs"
            if i > 200:
                name = f"{model_name}_{imgsize}imgsz_IP8M-H-SW-continuous"
                i=2000
                print("Resuming the continuous model")
                
            resume_flag = False
            if os.path.exists(os.path.join(f"{project}/{name}/weights/last.pt")):
                model = YOLO(f"{project}/{name}/weights/last.pt") 
                resume_flag = True
                print(f"Resuming {project}/{name} model")
            else:
                
                model = YOLO(f"yolo{model_name}.pt")  # load a pretrained model (recommended for training)
            
            results = model.train(resume=resume_flag, data="/data/mtmoore/school/CSiML_AI395T/final_project/dataset/yolov8_dataset/data.yaml", \
                                  project=project, name=name, epochs=i, imgsz=imgsize, batch=batchsizes[imgsize], plots=True, val=True)
        #name = f"11m_{imgsize}imgsz_IP8M-H-SW-{i}epochs-val"
        #results_val = model.val(project=project, name=name, save_json=True, plots=True, imgsz=imgsize, batch=1)


# Resume Training

In [ ]:
# Train the model
os.chdir("/data/mtmoore/school/CSiML_AI395T/final_project/dataset/yolov8_dataset/")
project="/data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative"
imgsize=2880
epoch="continuous"
model_base="11m"
camera="IP8M-H-SW"
name=f"{model_base}_{imgsize}imgsz_{camera}-{epoch}"
model = YOLO(f"{project}/{name}/weights/last.pt") 
# Resume training
results = model.train(resume=True, 
                      data="/data/mtmoore/school/CSiML_AI395T/final_project/dataset/yolov8_dataset/data.yaml", \
                      project=project, \
                      name=name, \
                      epochs=2000, \
                      imgsz=imgsize, 
                      batch=1, 
                      plots=True, 
                      val=True)

# Separate Validation

In [4]:
os.chdir("/data/mtmoore/school/CSiML_AI395T/final_project/dataset/yolov8_dataset/")
project="/data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative"

for model_name in ["11s", "11m"]:
    for imgsize in [720, 1440, 2880]:
        for i in list(range(1, 25)) + list(range(25, 50, 5)) + list(range(60, 101, 10)):
            name = f"{model_name}_{imgsize}imgsz_IP8M-H-SW-{i}epochs"
            if not os.path.exists( os.path.join(project, name) ):
                print(f"no model dir {name}")
                continue

            model = YOLO(f"{project}/{name}/weights/best.pt") 

            name = f"{model_name}_{imgsize}imgsz_IP8M-H-SW-{i}epochs-val"
            if os.path.exists( os.path.join(project, name, "metrics.json") ):
                print(f"already completed {os.path.join(project, name)}")
                continue

            metrics = model.val(data="/data/mtmoore/school/CSiML_AI395T/final_project/dataset/yolov8_dataset/data.yaml", \
                                project=project, name=name, save_json=True, plots=True, imgsz=imgsize, batch=1)
            store_metrics = metrics.results_dict.copy()
            store_metrics['category_mAP50-95'] = metrics.box.maps.tolist()
            with open(os.path.join(project, name, "metrics.json"), "w") as o:
                o.write(json.dumps(store_metrics))

already completed /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11s_720imgsz_IP8M-H-SW-1epochs-val
already completed /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11s_720imgsz_IP8M-H-SW-2epochs-val
already completed /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11s_720imgsz_IP8M-H-SW-3epochs-val
already completed /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11s_720imgsz_IP8M-H-SW-4epochs-val
already completed /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11s_720imgsz_IP8M-H-SW-5epochs-val
already completed /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11s_720imgsz_IP8M-H-SW-6epochs-val
already completed /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11s_720imgsz_IP8M-H-SW-7epochs-val
already completed /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11s_720imgsz_IP8M-H-SW-8epochs-val


val: Scanning val/labels.cache... 98 images, 0 backgrounds, 0 corrupt: 100%|██████████| 98/98 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 98/98 [00:05<00:00, 19.40it/s]


                   all         98        205      0.642      0.439      0.451      0.186
Speed: 6.7ms preprocess, 29.8ms inference, 0.0ms loss, 3.3ms postprocess per image
Saving /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11m_1440imgsz_IP8M-H-SW-3epochs-val2/predictions.json...
Results saved to /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11m_1440imgsz_IP8M-H-SW-3epochs-val2
Ultralytics 8.3.29 🚀 Python-3.12.7 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 11904MiB)
YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


val: Scanning val/labels.cache... 98 images, 0 backgrounds, 0 corrupt: 100%|██████████| 98/98 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 98/98 [00:04<00:00, 22.13it/s]


                   all         98        205      0.663      0.441       0.49      0.182
Speed: 5.0ms preprocess, 26.7ms inference, 0.0ms loss, 2.3ms postprocess per image
Saving /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11m_1440imgsz_IP8M-H-SW-4epochs-val/predictions.json...
Results saved to /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11m_1440imgsz_IP8M-H-SW-4epochs-val
Ultralytics 8.3.29 🚀 Python-3.12.7 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 11904MiB)
YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


...

                   all         98        205      0.924      0.883      0.925      0.615
Speed: 109.5ms preprocess, 65.2ms inference, 0.0ms loss, 2.0ms postprocess per image
Saving /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11m_2880imgsz_IP8M-H-SW-80epochs-val/predictions.json...
Results saved to /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11m_2880imgsz_IP8M-H-SW-80epochs-val
Ultralytics 8.3.29 🚀 Python-3.12.7 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 11904MiB)
YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


val: Scanning val/labels.cache... 98 images, 0 backgrounds, 0 corrupt: 100%|██████████| 98/98 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 98/98 [00:19<00:00,  5.13it/s]


                   all         98        205      0.923      0.863      0.909      0.588
Speed: 108.8ms preprocess, 65.5ms inference, 0.0ms loss, 2.0ms postprocess per image
Saving /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11m_2880imgsz_IP8M-H-SW-90epochs-val/predictions.json...
Results saved to /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11m_2880imgsz_IP8M-H-SW-90epochs-val
Ultralytics 8.3.29 🚀 Python-3.12.7 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 11904MiB)
YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


val: Scanning val/labels.cache... 98 images, 0 backgrounds, 0 corrupt: 100%|██████████| 98/98 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 98/98 [00:19<00:00,  5.16it/s]


                   all         98        205      0.923      0.823      0.897      0.595
Speed: 109.4ms preprocess, 65.3ms inference, 0.0ms loss, 2.0ms postprocess per image
Saving /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11m_2880imgsz_IP8M-H-SW-100epochs-val/predictions.json...
Results saved to /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11m_2880imgsz_IP8M-H-SW-100epochs-val
